In [1]:
import os
import mitsuba as mi

output_data_folder = 'data'
scene_file = 'scenes/cornell-box/scene.xml'

mi.set_variant('scalar_rgb')
scene = mi.load_file(scene_file)

[mitsuba] Warning: Couldn't import the ipywidgets package. Installing this package is required for the system to properly log messages and print in Jupyter notebooks!


In [9]:
# clear data if necessary
%rm -r data

In [10]:
# create data folder if necessary
os.makedirs(output_data_folder, exist_ok=True)

In [13]:
params = mi.traverse(scene)
params['logfile'] = 'data/gnn_file_2.path'
params.update();

In [14]:
mi.render(scene, spp=10)

TensorXf(shape=(64, 64, 3))

In [7]:
from mignn.container import SimpleLightGraphContainer
light_graphs1 = SimpleLightGraphContainer.fromfile('data/gnn_file_1.path', scene_file, verbose=True)
light_graphs2 = SimpleLightGraphContainer.fromfile('data/gnn_file_2.path', scene_file, verbose=True)
print(f'{light_graphs1}')
print(f'{light_graphs2}')

Load of `data/gnn_file_1.path` in progress: 100.00%
Load of `data/gnn_file_2.path` in progress: 100.00%
SimpleLightGraphContainer: [n_keys: 4096, n_graphs: 40960, n_nodes: 138231 (duplicate: 0), n_connections: 97271 (built: 0)]
SimpleLightGraphContainer: [n_keys: 4096, n_graphs: 40960, n_nodes: 138231 (duplicate: 0), n_connections: 97271 (built: 0)]


In [8]:
from mignn.manager import LightGraphManager
merged_graph_container = LightGraphManager.fusion([light_graphs1, light_graphs2])
print(merged_graph_container)

SimpleLightGraphContainer: [n_keys: 4096, n_graphs: 81920, n_nodes: 276462 (duplicate: 0), n_connections: 194542 (built: 0)]


In [ ]:
merged_graph_container.build_connections(n_graphs=10, n_nodes_per_graphs=2, n_neighbors=5, verbose=True)

In [ ]:
print(merged_graph_container)